# Libraries

In [234]:
import pandas as pd
import numpy as np
import ast
import plotly.express as px
import re
from sklearn.preprocessing import MultiLabelBinarizer
from selenium import webdriver
import datetime

# Import Data

In [3]:
booking = pd.read_csv('../csvs/booking_all.csv')

In [190]:
flights = pd.read_csv('../csvs/DataM.csv',sep = ';')

# Cleaning

## Booking

In [161]:
#remove na values
booking[~booking['price'].isna()].shape[0]/booking.shape[0]
booking = booking.drop_duplicates()
booking_clean = booking[~booking['price'].isna()]

In [162]:
booking_clean['price_int'] = booking_clean['price'].str.replace('€','')
booking_clean['price_int'] = booking_clean['price_int'].str.replace(',','')
booking_clean['price_int'] = booking_clean['price_int'].astype(int)
booking_clean['previous_price_int'] = booking_clean['previous_price'].str.replace('€','').str.replace(',','').astype(float)
booking_clean['location_score_float'] = booking_clean['location_score'].str.replace('Location','').astype(float)
booking_clean['score_float'] = booking_clean['score'].astype(float)
booking_clean['n_reviews_float'] = booking_clean['n_reviews'].str.extract('(\d*,*\d+)')
booking_clean['n_reviews_float'] = booking_clean['n_reviews_float'].str.replace(',','').astype(float)

C:\Users\antoi\AppData\Local\Temp\ipykernel_8676\3596275480.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  booking_clean['price_int'] = booking_clean['price'].str.replace('€','')
C:\Users\antoi\AppData\Local\Temp\ipykernel_8676\3596275480.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  booking_clean['price_int'] = booking_clean['price_int'].str.replace(',','')
C:\Users\antoi\AppData\Local\Temp\ipykernel_8676\3596275480.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [163]:
booking_clean = pd.get_dummies(booking_clean,columns=['property_type','score_label'],prefix = ['property','score'])
booking_clean[['property_type','score_label']] = booking[['property_type','score_label']]


In [164]:
booking_clean['appliances-list'] = booking_clean['appliances'].str.split(',')
appliances_onehot = pd.get_dummies(booking_clean['appliances-list'].apply(pd.Series).stack(dropna=False),prefix = 'appliances',dummy_na = True).groupby(level=0).sum()
booking_clean = pd.concat([booking_clean,appliances_onehot],axis = 1)
booking_clean['appliances_nan'] = booking_clean['appliances_nan'].apply(lambda na: 1 if na==np.max(booking_clean['appliances_nan']) else 0)

In [165]:
booking_clean['facilities-list'] = booking_clean['popular_facilities'].str.split(',')
facilities_onehot = pd.get_dummies(booking_clean['facilities-list'].apply(pd.Series).stack(dropna=False),prefix = 'facility',dummy_na = True).groupby(level=0).sum()
booking_clean = pd.concat([booking_clean,facilities_onehot],axis = 1)
booking_clean['facility_nan'] = booking_clean['facility_nan'].apply(lambda na: 1 if na==np.max(booking_clean['facility_nan']) else 0)

In [169]:
# undummify size
undummy_cols = [col for col in booking_clean.columns if re.match('appliances_(?!size)',col)]

sizes = booking_clean[undummy_cols]
booking_clean['appliances_size'] = sizes.idxmax(axis=1)
booking_clean['appliances_size'] = booking_clean['appliances_size'].str.extract('appliances_(\d+) m²')
booking_clean['appliances_size'] = booking_clean['appliances_size'].astype(float)

size_cols = [col for col in booking_clean.columns if re.match('appliances_\d+ m²',col)]
booking_clean = booking_clean.drop(columns=size_cols)

title                     object
price                     object
previous_price            object
location_score            object
address                   object
                          ...   
facility_Yoga classes      uint8
facility_iPad              uint8
facility_iPod dock         uint8
facility_nan               int64
appliances_size          float64
Length: 395, dtype: object

## Flights

In [198]:
flights = pd.read_csv('../csvs/DataM.csv',sep = ';')

In [199]:
flights

,index,TravelPrice,allerdepart,allerarrivee,retourdepart,retourarrivee,allercompanie,retourcompanie,allertimetravel,retourtimetravel,allerescale,retourescales,allerdeparth,allerarriveeh,retourdeparth,retourarriveeh
0,0,"112,99","Orly (ORY), Paris","Adolfo Suárez M... (MAD), Madrid","Adolfo Suárez M... (MAD), Madrid","Orly (ORY), Paris",· Iberia,· Iberia,2 h 05 min,2 h 05 min,direct,direct,Orly (ORY),Paris,NaN,NaN
1,1,"116,71","Adolfo Suárez M... (MAD), Madrid","Orly (ORY), Paris","Orly (ORY), Paris","Adolfo Suárez M... (MAD), Madrid",· Iberia,· Iberia,2 h 05 min,2 h 05 min,direct,direct,Adolfo Suárez M... (MAD),Madrid,NaN,NaN
2,2,"112,99","Orly (ORY), Paris","Adolfo Suárez M... (MAD), Madrid","Adolfo Suárez M... (MAD), Madrid","Orly (ORY), Paris",· Iberia,· Iberia,2 h 05 min,2 h 05 min,direct,direct,Orly (ORY),Paris,NaN,NaN
3,3,"116,71","Adolfo Suárez M... (MAD), Madrid","Orly (ORY), Paris","Orly (ORY), Paris","Adolfo Suárez M... (MAD), Madrid",· Iberia,· Iberia,2 h 05 min,2 h 05 min,direct,direct,Adolfo Suárez M... (MAD),Madrid,NaN,NaN
4,4,"116,71","Orly (ORY), Paris","Adolfo Suárez M... (MAD), Madrid","Adolfo Suárez M... (MAD), Madrid","Orly (ORY), Paris",· Iberia,· Iberia,2 h 05 min,2 h 05 min,direct,direct,Orly (ORY),Paris,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,390,"760,02","Charles De Gaul... (CDG), Paris","Adolfo Suárez M... (MAD), Madrid","Adolfo Suárez M... (MAD), Madrid","Charles De Gaul... (CDG), Paris",· Air France,· Air France,2 h 10 min,2 h 10 min,direct,direct,Charles De Gaul... (CDG),Paris,NaN,NaN
391,391,"779,65","Adolfo Suárez M... (MAD), Madrid","Charles De Gaul... (CDG), Paris","Charles De Gaul... (CDG), Paris","Adolfo Suárez M... (MAD), Madrid",· Klm Royal Dutch Airlines,· Air France,5 h 05 min,2 h 10 min,1 escale,direct,Adolfo Suárez M... (MAD),Madrid,NaN,NaN
392,392,"800,28","Charles De Gaul... (CDG), Paris","Adolfo Suárez M... (MAD), Madrid","Adolfo Suárez M... (MAD), Madrid","Orly (ORY), Paris",· Klm Royal Dutch Airlines,· Klm Royal Dutch Airlines,6 h 30 min,4 h 55 min,1 escale,1 escale,Charles De Gaul... (CDG),Paris,NaN,NaN
393,393,"818,88","Adolfo Suárez M... (MAD), Madrid","Orly (ORY), Paris","Charles De Gaul... (CDG), Paris","Adolfo Suárez M... (MAD), Madrid",· ITA Airways,· ITA Airways,5 h 30 min,8 h 40 min,1 escale,1 escale,Adolfo Suárez M... (MAD),Madrid,NaN,NaN


In [193]:
flights = flights.drop(columns = 'index')

In [200]:
flights['price_float'] = flights['TravelPrice'].str.replace(',','.')
flights['price_float'] = flights['price_float'].astype(float)
flights['price_float']

0      112.99
1      116.71
2      112.99
3      116.71
4      116.71
        ...  
390    760.02
391    779.65
392    800.28
393    818.88
394    910.65
Name: price_float, Length: 395, dtype: float64

In [213]:
flights['allercompanie'] = flights['allercompanie'].str[2:]
flights['retourcompanie'] = flights['retourcompanie'].str[2:]

In [249]:
def n_escales(esc):
    if esc == 'direct':
        return 0
    else:
        try:
            n_escale = int(re.search('(\d+) escale(s)?',esc).groups()[0])
        except:
            n_escale = None
        return n_escale

In [253]:
flights['aller_n_escale'] = flights['allerescale'].apply(lambda esc:n_escales(esc))
flights['retour_n_escale'] = flights['retourescales'].apply(lambda esc:n_escales(esc))

In [248]:
re.search('(\d+) escale(s)?','2 escales').groups()

('2', 's')

In [251]:
flights['aller_n_escale']

0      0
1      0
2      0
3      0
4      0
      ..
390    0
391    1
392    1
393    1
394    1
Name: aller_n_escale, Length: 395, dtype: int64

# Visualisations

In [117]:
def plot_avg(data,groupcol,aggcol,min = 0,max = None,width = 800,height = 400,title = 'title'):
    if max == None:
       max =  np.unique(data[groupcol]).shape[0] -1

    df_avg = data[[groupcol,aggcol]].groupby(groupcol).mean().sort_values(by = aggcol).reset_index()[min:max]


    fig = px.bar(df_avg,x = aggcol,y = groupcol,orientation='h',width = width,height=height,title=title)
   #  fig.update_layout(yaxis = dict(tickmode = 'linear'))
    return fig.show()

In [66]:
def plot_counts(data,col,min = 0,max = None,width = 800,height = 400,title = 'title'):
    if max == None:
       max =  np.unique(data[col]).shape[0] -1

    df_counts = pd.DataFrame(data[col].value_counts()).reset_index().rename(columns = {col:f'{col}_counts','index':col}).sort_values(by = f'{col}_counts')
    fig = px.bar(df_counts,x = f'{col}_counts',y=col,orientation='h',width = width,height=height,title=title)
    fig.update_layout(yaxis = dict(tickmode = 'linear'))
    return fig.show()

In [114]:
def plot_scatter(data,col1,col2,col3 = None,x_limit = None,y_limit=None,names = None,width = 800,height = 400,title = 'title',
col1_name = None,col2_name = None,col3_name = None):
    df_scatter = data[[col1,col2,col3]]
    if x_limit != None:
        df_scatter = df_scatter[df_scatter[col1]<=x_limit]
    if y_limit != None:
        df_scatter = df_scatter[df_scatter[col2]<=y_limit]
    if names != None:
        df_scatter = df_scatter[df_scatter[col3].isin(names)]
    
    if col1_name == None:
        col1_name = col1
    if col2_name == None:
        col2_name = col2
    if col3_name == None:
        col3_name = col3

    fig = px.scatter(df_scatter,col1,col2,color = col3,width=width,height = height,title = title,
    labels={col1:col1_name, col2:col2_name, col3:col3_name})
    return fig.show()

In [56]:
plot_avg(booking_clean,'property_type','price_int',width = 800,height=400,title='Average price per type of stay')

In [57]:
plot_avg(booking_clean,'property_type','location_score_float',width = 800,height=400,title='Average location score per stay')

In [67]:
plot_counts(booking_clean,'property_type',title='Number of property types')

In [113]:
plot_scatter(booking_clean,'price_int','score_float','property_type',x_limit = 1000,names=['Guesthouse','Hotel','Apartment'],
title = 'Notes en fonction du prix et du type de sejour',col1_name='prix',col2_name='note',col3_name='type')

prix


In [118]:
plot_avg(booking_clean,'property_type','n_reviews_float',title = 'Average number of reviews per type')

In [120]:
plot_scatter(booking_clean,'n_reviews_float','score_float','property_type',x_limit = 5000,names=['Guesthouse','Hotel','Apartment'],
title = 'Notes en fonction du prix et du type de sejour',col1_name='number of reviews',col2_name='note',col3_name='type')

# Find hotel

In [39]:
booking_clean['appliances_100 m²\r\nSize']

0        0
1        0
2        0
3        0
4        0
        ..
20459    0
20460    0
20461    0
20462    0
20463    0
Name: appliances_98 m²\r\nSize, Length: 19812, dtype: uint8

In [45]:
size = 'appliances_100 m²\r\nSize'
size.replace('\r\n',' ')

'appliances_100 m² Size'

'la lala 135 1039 921'